In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import shapefile
import pyproj
from matplotlib.path import Path

In [2]:
#! /usr/bin/env python3

# data libraries
import sys
import os
import datetime as dt  # Python standard library datetime  module
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
from netCDF4 import Dataset,num2date  # http://code.google.com/p/netcdf4-python/
import pandas as pd
import geopandas as gpd
from pyproj import Proj, transform

# plotting libraries
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.style as style
from matplotlib import cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point 
import cartopy.mpl.ticker as cticker
import glob


In [3]:
def getPolyCoords(row, geom, coord_type):
    """
    Returns the coordinates ('x|y') of edges/vertices of a Polygon/others
    Args:
    - row: the row object from a geodataframe; i.e.   df.loc[1]
    - geom: the name of "geometry" column, usually "geometry"
    - coord_type: 'x', or 'y'
    ...
    Valid geometrie types 'Polygon', 'Point', 'LineString', 'MultiLineString', 'MultiPolygon'
    """

    # Parse the geometries and grab the coordinate
    geometry = row[geom]
    #print(geometry.type)

    if geometry.type=='Polygon':
        if coord_type == 'x':
            # Get the x coordinates of the exterior
            # Interior is more complex: xxx.interiors[0].coords.xy[0]
            return list( geometry.exterior.coords.xy[0] )
        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return list( geometry.exterior.coords.xy[1] )

    if geometry.type in ['Point', 'LineString']:
        if coord_type == 'x':
            return list( geometry.xy[0] )
        elif coord_type == 'y':
            return list( geometry.xy[1] )

    if geometry.type=='MultiLineString':
        all_xy = []
        # updated code, using .geoms
        for ea in geometry.geoms:
            if coord_type == 'x':
                all_xy.append(list( ea.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.xy[1] ))
        return all_xy

    if geometry.type=='MultiPolygon':
        all_xy = []
        for ea in geometry.geoms:
            if coord_type == 'x':
                all_xy.append(list( ea.exterior.coords.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.exterior.coords.xy[1] ))
        return all_xy

    else:
        # Finally, return empty list for unknown geometries
        return []

In [4]:
shelves = list()
inds = list()

In [5]:
def create_shelf_path(path):

    shelf = gpd.read_file(path) 
    shelf.crs = "EPSG:3031"
    shelf = shelf.to_crs(epsg=4326)

    shelf_lon = getPolyCoords(shelf.loc[0], 'geometry', 'x')
    shelf_lat = getPolyCoords(shelf.loc[0], 'geometry', 'y')

    shelf_list = np.vstack([shelf_lat,shelf_lon]).T
    shelf_path = Path(shelf_list)

    return shelf_lon,shelf_lat,shelf_path

In [6]:
def shelf_lat_lon(path):
    
    #Reading in Datafiles
    fname = path
    datain = Dataset(fname, 'r')
    
    head_tail = os.path.split(path) #makes the path a variable
    mystring = str(head_tail[1]) #turns the variable into a string
    x = mystring.split(".", 5)#parsing the model name by periods
   
    #Creating Variables
    lat = datain.variables['latitude'][:] #latitude
    lon = datain.variables['longitude'][:] #longitude
    
    shelf_lat_model, shelf_lon_model = np.meshgrid(lat,lon)
    
    return shelf_lat_model,shelf_lon_model,x

In [7]:
def shelf_indicies(shelf_path,shelf_lat_model,shelf_lon_model):
    
    inds = shelf_path.contains_points(np.vstack([shelf_lat_model.ravel(), shelf_lon_model.ravel()]).T)
    
    return inds
    

In [8]:
shelf_paths = pd.read_csv('/raid01/mafields/tas/MODELS_filtered/ssp585/path_data/shape_files/shapefile_paths.csv')
reanalysis_paths = pd.read_csv('/raid01/mafields/tas/MODELS_filtered/ssp585/path_data/reanalysis/reanalysis_monthly_paths.csv')
monthly_paths = pd.read_csv('/raid01/mafields/tas/MODELS_filtered/ssp585/path_data/monthly/monthly_paths.csv')
reanalysis_paths = reanalysis_paths.drop([3])
monthly_paths = monthly_paths.drop([3])
print(monthly_paths['Shelf Path'][4])

/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/baudouin/monthly


In [9]:
def finding_indicies(path,path1):
    shelf_name = list()
    shelf_inds = list()
    shelf_model_lons = list()
    shelf_model_lats = list()
    shelf_lon, shelf_lat, shelf_path = create_shelf_path(path)
    
    for file in glob.iglob(f'{path1}/*.nc'): #iterating through files in path specified
        shelf_lat_model,shelf_lon_model,file_name = shelf_lat_lon(file)
        shelf_model_lons.append(shelf_lon_model)
        shelf_model_lats.append(shelf_lat_model)
        inds = shelf_indicies(shelf_path,shelf_lat_model,shelf_lon_model)
    
    shelf_name.append(file_name[4])
    shelf_inds.append(inds)
        
    return shelf_name,shelf_inds,shelf_lon,shelf_lat,shelf_model_lons,shelf_model_lats
        

In [211]:
shelf_name,shelf_inds,lon,lat,lon_model,lat_model = finding_indicies(shelf_paths['Shelf Shape File Path'][47],monthly_paths['Shelf Path'][31])
shelves.append(shelf_name)
inds.append(shelf_inds)

In [212]:
print(shelves)

[['abbot'], ['amery'], ['atka'], ['baudouin'], ['borchgrevink'], ['bruntStancomb'], ['congerGlenzer'], ['cook'], ['cosgrove'], ['crosson'], ['dotson'], ['drygalski'], ['ekstrom'], ['filchner'], ['fimbul'], ['georgeVI'], ['getz'], ['holmes'], ['jelbart'], ['land'], ['larsenB'], ['larsenC'], ['larsenD'], ['lazarev'], ['mariner'], ['mertz'], ['moscowU'], ['nansen'], ['nickerson'], ['nivl'], ['princeH'], ['quar'], ['rennick'], ['riiser'], ['ronne'], ['ross'], ['ross'], ['shackleton'], ['stange'], ['sulzberger'], ['thwaites'], ['totten'], ['tucker'], ['venable'], ['vigrid'], ['west'], ['withrow'], ['pineIsland']]


In [225]:
shelf_indicies = pd.DataFrame(list(zip(shelves,inds)),columns=['Shelf Names','Indicies'])
os.makedirs('/raid01/mafields/tas/MODELS_filtered/ssp585/indicies', exist_ok=True)  
shelf_indicies.to_csv('/raid01/mafields/tas/MODELS_filtered/ssp585/indicies/shelf_indicies.csv') 